Hasta ahora solo un agente ha entrado en juego, pero ¿qué pasa si queremos aportar algo de diversidad? Esto es especialmente relevante cuando entran en juego procesos creativos u opiniones:

* Puede que queramos hacer un análisis SEO
* Crear un informe sobre el progreso de un empleado
* Definir una política de trabajo híbrido

Y todas estas tareas **necesitarán experiencia humana**, pero podemos empezar con un buen borrador que aporte diferentes opiniones. Algo que normalmente requeriría varios talleres e implicaría diferentes roles, todo automatizado. La idea es realmente cautivadora.

## Frameworks Multiagente

[CrewAI](https://www.crewai.com/) fue creado con esta idea en mente: poder definir programáticamente una multitud, asignarles una tarea y permitirles discutir, evolucionar y refinar el resultado antes de presentarlo al usuario.

De manera similar, [Agno](https://app.agno.com/) ofrece la posibilidad de desplegar y probar aplicaciones multiagente gracias a su playground.

![agnomulti](../assets/images/agno_multi.png)

Podemos optar por LangChain si queremos tener un mayor control sobre la arquitectura que se está implementando.

![](https://langchain-ai.github.io/langgraph/concepts/img/multi_agent/architectures.png)

Más información en https://langchain-ai.github.io/langgraph/concepts/multi_agent/

Pero el punto principal de los frameworks anteriores es que facilitan la definición de la transferencia de tareas entre agentes y la interacción general entre ellos antes de proporcionar una respuesta final.

In [3]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

Necesitaremos equipar a nuestros investigadores con algunas herramientas, en particular herramientas de búsqueda web. Estas herramientas pueden personalizarse tanto como sea necesario para acotar las posibles respuestas.

Emplearemos [Tavily](https://www.tavily.com/) un buscador pensado para interacción vía API que encaja perfecto con nuestro uso.

In [4]:
from agno.tools.tavily import TavilyTools

# Search engine
search = TavilyTools(
    include_answer=True,
    search_depth='basic',
    format='markdown',
    max_tokens=100
)

Un primer agente se encargará del análisis SEO mientras que el segundo agente optimizará el contenido. Si ambos colaboran, el impacto de la creación de contenido vendrá dado por la capacidad del primer agente, el analista SEO, para buscar y refinar el contexto identificando palabras clave.

In [5]:
from agno.agent import Agent
from agno.models.google import Gemini

seo_analyst = Agent(
    name="SEO Analyst",
    role="Discovers the keywords needed to boost the position of the site to be improved.",
    model=Gemini(id="gemini-2.5-flash", temperature=0),
    tools=[search],
    instructions="""
        Backstory:
        
        You work in Spain, most of your requests come from Spanish companies so you prioritize the positioning in Spanish speaking searches.
        Your specialty is identifying keywords to better position the products highlighted by the company.
        You conduct research on competitors and search results to better differentiate the offering and be more competitive.

        Response format:
        Always include sources.
    """,
    add_datetime_to_instructions=True,
)

strategist = Agent(
    name="Content strategist",
    role="Creates attractive and engaging content to improve SEO positioning.",
    model=Gemini(id="gemini-2.5-flash", temperature=0.6),
    tools=[search], # It might not need it but just in case
    instructions="""
        Backstory:
        
        You are a renowned Content Strategist, known for your insightful and engaging articles.
        Your specialty is transforming complex concepts into captivating narratives with high SEO impact.

        Response format:
        Always include sources.
    """,
    add_datetime_to_instructions=True,
)

Necesitamos agruparlos y asignarles la tarea a realizar. Esto requiere un **gestor** con cierta capacidad de razonamiento para utilizar eficientemente los recursos disponibles. Para ello se necesita:

* Capacidades de razonamiento
* Algunas instrucciones
* Saber cuándo finalizar el trabajo

In [6]:
from agno.team.team import Team
from agno.tools.reasoning import ReasoningTools

seon_team = Team(
    name="SEO improving content creation team",
    mode="coordinate",
    model=Gemini(id="gemini-2.5-flash", temperature=0.3),
    members=[seo_analyst, strategist],
    tools=[ReasoningTools(add_instructions=True)],
    instructions=[
        "Collaborate to provide comprehensive SEO improving insights",
        "Consider both fundamental analysis and market sentiment",
        "Use an engaging language to provide guidance to the user asking the questions",
        "Present findings in a structured, easy-to-follow format",
        "Only output the final consolidated response, not individual agent responses",
    ],
    markdown=True,
    show_members_responses=True,
    enable_agentic_context=True,
    add_datetime_to_instructions=True,
    success_criteria="The team has provided a complete analysis on SEO positioning and the keywords and modifications required to boost the position of the site being asked about.",
)

In [7]:
seon_team.print_response(
"""The Bridge School is providing upskiller courses dedicated to improving the abilities of professionals with a special focus on Data Science and Artificial intelligence.
How would you recommend improving the text and visuals of that section fo the site to achieve that goal?
""",
    stream=True,
    show_full_reasoning=True,
    stream_intermediate_steps=True,
)

Output()

Todo este proceso puede ocurrir en cualquier frontal capaz de recibir interacciones de este tipo, con lo que no nos limitemos por el ejercicio que aquí indicamos ya que existen frameworks específicos para aplicaciones frontales y los MCP nos permiten abstraer los frontales, LLMs y herramientas a emplear en nuestro proceso.